In [1]:
# @title Install required libraries

!pip install -q faiss-cpu langchain langchain-community langchain-cohere cohere==5.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [2]:
# @title Imports

import requests
import numpy as np
import pandas as pd
from google.colab import userdata
from IPython.display import display

# LangChain & LangChain Community
import faiss
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS

# Cohere integrations
from langchain_cohere import ChatCohere
from langchain_cohere import CohereEmbeddings

In [3]:
# @title Download excel files

headers = {
    'User-Agent': 'microsoft bill.gates@microsoft.com',
    'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
}

response = requests.get("https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/Financial_Report.xlsx", headers=headers)
with open("/content/Apple_Financial_2021.xlsx", 'wb') as f:
    f.write(response.content)

response = requests.get("https://www.sec.gov/Archives/edgar/data/0001682852/000168285225000022/Financial_Report.xlsx", headers=headers)
with open("/content/Moderna_Financial_2025.xlsx", 'wb') as f:
    f.write(response.content)

response = requests.get("https://www.sec.gov/Archives/edgar/data/0000064803/000006480324000007/Financial_Report.xlsx", headers=headers)
with open("/content/CVS_Health_Financial_2024.xlsx", 'wb') as f:
    f.write(response.content)

response = requests.get("https://www.sec.gov/Archives/edgar/data/0001318605/000095017023001409/Financial_Report.xlsx", headers=headers)
with open("/content/Tesla_Financial_2023.xlsx", 'wb') as f:
    f.write(response.content)



#XXXXXXXXXXXXXXXXXXXXXXXXXX TO DOWNLOAD COMPLETE DATA XXXXXXXXXXXXXXXXXXXXXXXXXX

# import os
# import ast
# import shutil
# from google.colab import files

# data_dir = "data_files"
# os.makedirs(data_dir, exist_ok=True)

# uploaded = files.upload()

# for filename in uploaded.keys():
#     print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')
#     shutil.move(filename, os.path.join(data_dir, filename))
#     print(f'Moved "{filename}" to "{data_dir}/"')

# # To access the uploaded file:
# for filename in uploaded.keys():
#     print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

# path_tickers = '/content/data_files/sec_tickers.txt'
# path_ciks = '/content/data_files/sec_ciks.txt'
# path_acc_nos = '/content/data_files/sec_acc_nos.txt'

# tickers = dict()
# ciks = dict()
# acc_nos = dict()

# with open(path_tickers, 'r') as f:
#     data = f.read()
#     tickers = ast.literal_eval(data)

# with open(path_ciks, 'r') as f:
#     data = f.read()
#     ciks = ast.literal_eval(data)

# with open(path_acc_nos, 'r') as f:
#     data = f.read()
#     acc_nos = ast.literal_eval(data)

# excel_dir = "excel_files"
# os.makedirs(excel_dir, exist_ok=True)
# os.chdir(excel_dir)

# # To store the files which were not able to download
# files_not_downloaded = []

# for cik in acc_nos:
#     for acc_no in acc_nos[cik]:
#         excel_link = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_no.replace('-','')}/Financial_Report.xlsx"
#         company_name = tickers[ciks[cik]]

#         try:
#             response = requests.get(excel_link, headers=headers)
#             response.raise_for_status()
#             with open(f"{company_name}_Financial_20{acc_no.split('-')[1]}.xlsx", 'wb') as f:
#                 f.write(response.content)
#                 print(f"Downloaded {company_name}_Financial_20{acc_no.split('-')[1]}.xlsx")
#         except requests.exceptions.RequestException as e:
#             print(f"Error downloading {company_name}_Financial_20{acc_no.split('-')[1]}.xlsx: {e}")
#             files_not_downloaded.append(f"{company_name}_Financial_20{acc_no.split('-')[1]}.xlsx")

# os.chdir('/content')

# # Get all the file paths for easier processing of excel files
# file_list = []
# for root, _, files in os.walk('/content/excel_files/'):
#     for file in files:
#         # Construct the full file path
#         file_list.append(os.path.join(root, file))

# print(len(file_list))


In [4]:
# @title excel_to_df(excel_file_path: str) - Convert excel data to dataframe
def excel_to_df(excel_file_path: str):

    # Description of sheet names that need to be read from excel files
    sheet_name_desc1 = 'consolidated statements of oper'
    sheet_name_desc2 = 'consolidated statements of incom'

    try:
        matched_sheet = ''
        excel_content = pd.ExcelFile(excel_file_path)
        for sheet_name in excel_content.sheet_names:
            if sheet_name_desc1 in sheet_name.lower() or sheet_name_desc2 in sheet_name.lower():
              matched_sheet = sheet_name
              break
        df = pd.read_excel(excel_file_path, sheet_name = matched_sheet, engine = 'openpyxl')
        files_parsed.append(excel_file_path)

    except Exception as e:
        df = f'Could not read the excel file {excel_file_path}!'
        files_not_parsed.append(excel_file_path)

    return df

In [13]:
# @title def process_df_values(df: pd.dataframe, share_multiplier: int, usd_multiplier: int) -> Correct the values in dataframe according to share and usd multipliers

def process_df_values(df: pd.DataFrame, share_multiplier: int, usd_multiplier: int) -> pd.DataFrame:
    pre_string = ''
    nan_index, text_index = [], []

    # Some common terms to be taken care of when converting dataframe values
    common_averages = ["in share", "average share", "average common share"]
    usd_terms = ["in usd", "in dollars", "income per share"]

    for index, row in df.iterrows():
        try:
            # Get row description for appropriate value conversion
            row_desc = row.loc['Description'].lower()
            if row_desc[-1] == ":":
                row_desc = row_desc[:-1]
        except:
            continue

        if row.isnull().all():
          pre_string = ''
          nan_index.append(index)

        elif row.iloc[2:].isnull().all():
          if index - 1 in nan_index:
            pre_string = pre_string + " | " + row_desc
          else:
            pre_string = row_desc
          nan_index.append(index)

        else:
            try:
                # If value can't be converted to float, then skip that row
                float(row.iloc[2])

                if pre_string:
                    df.iloc[index, 0] = pre_string

                for col in df.columns[2:]:
                    val = row[col]
                    if any(common_average in row_desc for common_average in common_averages) or any(common_average in pre_string for common_average in common_averages):
                        val = float(val) * share_multiplier
                        val = str(val) + " shares"
                    else:
                        if all(usd_term not in row_desc and usd_term not in pre_string for usd_term in usd_terms):
                            val = float(val) * usd_multiplier
                            if abs(val) >= 1000000000:
                                val = str(val/1000000000) + " billion"
                            elif abs(val) >= 1000000:
                                val = str(val/1000000) + " million"

                        val = "USD " + str(val)

                    df.at[index, col] = val
            except:
                text_index.append(index)
                continue

    # Drop rows that contain text and not numbers
    if text_index:
      df = df.drop(text_index)

    # Drop empty rows
    df = df.dropna(subset=df.columns[1:])

    return df.reset_index(drop = True)

In [6]:
# @title def process_dataframe(df: pd.dataframe) -> Set column headings and find share and usd multipliers

def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:

    # Identify multipliers from the first row of the sheet
    multiplier_identifier = df.columns[0].lower()

    share_multiplier = 1
    if 'shares in billion' in multiplier_identifier:
        share_multiplier = 1000000000
    elif 'shares in million' in multiplier_identifier:
        share_multiplier = 1000000
    elif 'shares in thousand' in multiplier_identifier:
        share_multiplier = 1000

    usd_multiplier = 1
    if '$ in billion' in multiplier_identifier:
        usd_multiplier = 1000000000
    elif '$ in million' in multiplier_identifier:
        usd_multiplier = 1000000
    elif '$ in thousand' in multiplier_identifier:
        usd_multiplier = 1000

    # Rename columns with None heading for proper column renaming
    df.columns = [f'new_column_{i}' if col is None else col for i, col in enumerate(df.columns)]

    # Replace empty dataframe values with NaN for better processing
    df = df.map(lambda x: np.nan if (x == '' or x is None or (isinstance(x, str) and x.strip() == "")) else x)

    # Delete columns with very less unique values as they do not contain good data
    for column in df.columns:
        if df[column].nunique() < 5:
            df = df.drop(columns=[column])

    # Rename columns with their respective years
    df = df.rename(columns = {df.columns[0]: "Description",
                              df.columns[1]: int(df.iloc[0,1][-4:]),
                              df.columns[2]: int(df.iloc[0,2][-4:]),
                              df.columns[3]: int(df.iloc[0,3][-4:])})

    # Remove first row as its not required and add a new column 'Category' to dataframe
    df = df[1:]
    df.insert(0, 'Category', '')
    df = df.reset_index(drop = True)      # To make sure indices are in sync

    return process_df_values(df = df, share_multiplier = share_multiplier, usd_multiplier = usd_multiplier)


In [7]:
# @title def df_to_document(df:  pd.DataFrame, company_name: str) -> Convert dataframe to langchain Documents
def df_to_document(df:  pd.DataFrame, company_name: str):
    documents = []

    # Convert each dataframe row to text for mebedding
    for index, row in df.iterrows():
      if row.iloc[0]:
        row_desc = f"For {company_name} {row.iloc[1]} in {row.iloc[0]} category for year"
      else:
        row_desc = f"For {company_name} {row.iloc[1]} for year"

      for column in df.columns[2:]:
        metadata = {"company": company_name, "year": column}
        documents.append(Document(page_content = f"{row_desc} {column} is {row[column]}.", metadata = metadata))

    return documents

In [14]:
# @title MAIN

file_list = ['/content/Apple_Financial_2021.xlsx', '/content/Moderna_Financial_2025.xlsx', '/content/CVS_Health_Financial_2024.xlsx', '/content/Tesla_Financial_2023.xlsx']
files_parsed, files_not_parsed = [], []

docs = []

# Convert each excel to langchain documents for embedding
for excel_file_path in file_list:

    company_name = (excel_file_path.split('/')[-1]).split('_')[0]
    df = excel_to_df(excel_file_path = excel_file_path )
    if not isinstance(df, pd.DataFrame):
        files_not_parsed.append(excel_file_path)
        continue

    files_parsed.append(excel_file_path)
    df = process_dataframe(df = df)

    print(excel_file_path)
    display(df)
    print("\n\n")
    docs.extend(df_to_document(df = df, company_name = company_name))


/content/Apple_Financial_2021.xlsx


,Category,Description,2021,2020,2019
0,,Net sales,USD 365.817 billion,USD 274.515 billion,USD 260.174 billion
1,,Cost of sales,USD 212.981 billion,USD 169.559 billion,USD 161.782 billion
2,,Gross margin,USD 152.836 billion,USD 104.956 billion,USD 98.392 billion
3,operating expenses,Research and development,USD 21.914 billion,USD 18.752 billion,USD 16.217 billion
4,operating expenses,"Selling, general and administrative",USD 21.973 billion,USD 19.916 billion,USD 18.245 billion
5,operating expenses,Total operating expenses,USD 43.887 billion,USD 38.668 billion,USD 34.462 billion
6,operating expenses,Operating income,USD 108.949 billion,USD 66.288 billion,USD 63.93 billion
7,operating expenses,"Other income/(expense), net",USD 258.0 million,USD 803.0 million,USD 1.807 billion
8,operating expenses,Income before provision for income taxes,USD 109.207 billion,USD 67.091 billion,USD 65.737 billion
9,operating expenses,Provision for income taxes,USD 14.527 billion,USD 9.68 billion,USD 10.481 billion





/content/Moderna_Financial_2025.xlsx


,Category,Description,2024,2023,2022
0,revenue,Total revenue,USD 3.236 billion,USD 6.848 billion,USD 19.263 billion
1,operating expenses,Cost of sales,USD 1.464 billion,USD 4.693 billion,USD 5.416 billion
2,operating expenses,Research and development,USD 4.543 billion,USD 4.845 billion,USD 3.295 billion
3,operating expenses,"Selling, general and administrative",USD 1.174 billion,USD 1.549 billion,USD 1.132 billion
4,operating expenses,Total operating expenses,USD 7.181 billion,USD 11.087 billion,USD 9.843 billion
5,operating expenses,(Loss) income from operations,USD -3.945 billion,USD -4.239 billion,USD 9.42 billion
6,operating expenses,Interest income,USD 425.0 million,USD 421.0 million,USD 200.0 million
7,operating expenses,"Other expense, net",USD -87.0 million,USD -124.0 million,USD -45.0 million
8,operating expenses,(Loss) income before income taxes,USD -3.607 billion,USD -3.942 billion,USD 9.575 billion
9,operating expenses,(Benefit from) provision for income taxes,USD -46.0 million,USD 772.0 million,USD 1.213 billion





/content/CVS_Health_Financial_2024.xlsx


,Category,Description,2023,2022,2021
0,revenues,Premiums,USD 99.192 billion,USD 85.33 billion,USD 76.132 billion
1,revenues,Net investment income,USD 1.153 billion,USD 838.0 million,USD 1.199 billion
2,revenues,Total revenues,USD 357.776 billion,USD 322.467 billion,USD 292.111 billion
3,operating costs,Cost of products sold,USD 217.098 billion,USD 196.892 billion,USD 175.803 billion
4,operating costs,Health care costs,USD 86.247 billion,USD 71.073 billion,USD 64.188 billion
5,operating costs,Restructuring charges,USD 507.0 million,USD 0.0,USD 0.0
6,operating costs,Opioid litigation charges,USD 0.0,USD 5.803 billion,USD 0.0
7,operating costs,Loss on assets held for sale,USD 349.0 million,USD 2.533 billion,USD 0.0
8,operating costs,Store impairments,USD 0.0,USD 0.0,USD 1.358 billion
9,operating costs,Goodwill impairment,USD 0.0,USD 0.0,USD 431.0 million





/content/Tesla_Financial_2023.xlsx


,Category,Description,2022,2021,2020
0,revenues,Total revenues,USD 81.462 billion,USD 53.823 billion,USD 31.536 billion
1,cost of revenues,Total cost of revenues,USD 60.609 billion,USD 40.217 billion,USD 24.906 billion
2,cost of revenues,Gross profit,USD 20.853 billion,USD 13.606 billion,USD 6.63 billion
3,operating expenses,Research and development,USD 3.075 billion,USD 2.593 billion,USD 1.491 billion
4,operating expenses,"Selling, general and administrative",USD 3.946 billion,USD 4.517 billion,USD 3.145 billion
5,operating expenses,Restructuring and other,USD 176.0 million,USD -27.0 million,USD 0.0
6,operating expenses,Total operating expenses,USD 7.197 billion,USD 7.083 billion,USD 4.636 billion
7,operating expenses,Income from operations,USD 13.656 billion,USD 6.523 billion,USD 1.994 billion
8,operating expenses,Interest income,USD 297.0 million,USD 56.0 million,USD 30.0 million
9,operating expenses,Interest expense,USD -191.0 million,USD -371.0 million,USD -748.0 million


In [23]:
# @title Embedding and Cohere Configuration

cohere_api_key = userdata.get('COHERE_API_KEY')
embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, model="embed-english-v3.0")

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 15})

llm = ChatCohere(cohere_api_key=cohere_api_key)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=False)

In [24]:
# @title Query 1 to Cohere

query = "Compare Moderna and Tesla's net income"
response = qa_chain.invoke({"query": query})

print("\nAnswer:")
print(response['result'])


Answer:
Based on the provided information, here’s a comparison of Moderna and Tesla's net income for the available years:

### **Moderna Net (Loss) Income:**
- **2024**: USD -3.561 billion (loss)  
- **2023**: USD -4.714 billion (loss)  
- **2022**: USD 8.362 billion (profit)  

### **Tesla Net Income:**
- **2022**: USD 12.556 billion (profit)  
- **2021**: USD 5.644 billion (profit)  
- **2020**: USD 721.0 million (profit)  

### **Key Observations:**
1. **Profitability Trends:**  
   - Moderna reported a significant profit in 2022, likely driven by COVID-19 vaccine sales, but shifted to losses in 2023 and 2024.  
   - Tesla has consistently reported profits since 2020, with a notable increase in 2022.  

2. **Scale of Income:**  
   - Tesla's net income in 2022 (USD 12.556 billion) is substantially higher than Moderna's peak profit in 2022 (USD 8.362 billion).  
   - Moderna's losses in 2023 and 2024 are significant, while Tesla has maintained profitability.  

3. **Industry Context

In [26]:
# @title Query 2 to Cohere

query = "What was CVS's net income in 2022?"
response = qa_chain.invoke({"query": query})

print("\nAnswer:")
print(response['result'])


Answer:
According to the provided information, CVS's net income in 2022 was USD 4.327 billion.


In [25]:
# @title Query 3 to Cohere

query = "What is income tax provision?"
response = qa_chain.invoke({"query": query})

print("\nAnswer:")
print(response['result'])


Answer:
The **income tax provision** is an estimate of the amount a company expects to pay in taxes on its earnings for a given period. It is recorded on the company’s income statement as an expense, reflecting the tax liability associated with its taxable income. The provision is calculated based on the company’s pretax income and the applicable tax rates, adjusted for any tax credits, deductions, or other factors.

Key points about income tax provision:

1. **Purpose**: It ensures that the company’s financial statements accurately reflect its tax obligations, aligning with accounting principles like GAAP (Generally Accepted Accounting Principles) or IFRS (International Financial Reporting Standards).

2. **Components**:
   - **Current Tax Provision**: Taxes payable on current-year income.
   - **Deferred Tax Provision**: Adjustments for temporary differences between accounting and tax rules, which may result in future tax liabilities or benefits.

3. **Calculation**: It is typically

In [27]:
# @title Query 4 to Cohere

query = "Compare the operating expenses across all companies"
response = qa_chain.invoke({"query": query})

print("\nAnswer:")
print(response['result'])


Answer:
Here’s a comparison of the **total operating expenses** across the companies based on the provided data:

### **Apple:**
- **2019:** USD 34.462 billion  
- **2020:** USD 38.668 billion  
- **2021:** USD 43.887 billion  

### **Tesla:**
- **2020:** USD 4.636 billion  
- **2021:** USD 7.083 billion  
- **2022:** USD 7.197 billion  

### **Moderna:**
- **2022:** USD 9.843 billion  
- **2023:** USD 11.087 billion  
- **2024:** USD 7.181 billion  

### **CVS:**
- **2021:** USD 37.021 billion (Operating expenses)  
- **2022:** USD 38.212 billion (Operating expenses)  
- **2023:** USD 39.832 billion (Operating expenses)  

### **Key Observations:**
1. **Apple** has the highest operating expenses across all years, with a steady increase from 2019 to 2021.  
2. **CVS** follows Apple with significant operating expenses, though they are lower than Apple's.  
3. **Tesla** and **Moderna** have much lower operating expenses compared to Apple and CVS, with Tesla's expenses increasing from 20

In [28]:
# @title Query 5 to Cohere
query = "Compare moderna's profitability across years"
response = qa_chain.invoke({"query": query})

print("\nAnswer:")
print(response['result'])


Answer:
To compare Moderna's profitability across the years 2022, 2023, and 2024, we can analyze key financial metrics such as **Net Income**, **(Loss) Income from Operations**, and **(Loss) Earnings Per Share (EPS)**. Here’s a breakdown:

### **1. Net (Loss) Income in Operating Expenses Category**
- **2022**: USD 8.362 billion (profit)  
- **2023**: USD -4.714 billion (loss)  
- **2024**: USD -3.561 billion (loss)  

**Observation**: Moderna was profitable in 2022 but shifted to significant losses in 2023 and 2024, with the loss narrowing slightly in 2024 compared to 2023.

---

### **2. (Loss) Income from Operations**
- **2022**: USD 9.42 billion (profit)  
- **2023**: USD -4.239 billion (loss)  
- **2024**: USD -3.945 billion (loss)  

**Observation**: Similar to net income, Moderna was highly profitable in 2022 but incurred substantial operational losses in 2023 and 2024, with a slight improvement in 2024.

---

### **3. (Loss) Income Before Income Taxes**
- **2022**: USD 9.575 bi

In [33]:
# @title Setting up gradio

!pip -q install gradio

In [37]:
import gradio as gr

def answer_question(question):
    response = qa_chain.invoke(question)  # your LangChain QA chain
    return response["result"]

gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(label="Ask a question"),
    outputs=gr.Textbox(label="Answer"),
    title="Financial QA Assistant"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9892987535dd37092c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
